In [1]:
import csv
import sqlite3
import os
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
def preprocess_text_column(col:pd.Series) -> pd.Series:
    """
    Arregla texto de una columna que contiene caracteres especiales
    y espacios.
    """

    ## Remove special chars
    series = col.replace(r'[^\w\sáéíóúÁÉÍÓÚñÑ]', '', regex=True)

    ## Remove double white spaces
    series = series.replace(r'\s+', ' ', regex=True)
    series = series.str.strip()
    series = series.str.lower()

    ## Capitalize what columns cointains. "valle del cauca" -> "Valle Del Cauca"
    series = series.apply(lambda t: ' '.join([l.capitalize() for l in t.split(' ')]))
    return series


def preprocess_read_reps_data(data_path:str) -> pd.DataFrame:
    """
    Función para leer información descargada de REPS. En las bases de datos de "Sedes"
    y "Servicios" tienen la particularidad de que al hacer la lectura con pandas se 
    corren los encabezados una columna. 
    """
    
    ## Read dataframe from data_path
    df = pd.read_csv(data_path, sep=';', encoding='ISO-8859-1', on_bad_lines='skip', dtype=str)
    
    ## Alocate first colum from index
    df = df.reset_index(names='wildcard')

    ## Clean columns names and assign correct order
    cols = [c.upper().strip() for c in df.columns]

    ## Move WILCARD column to last position
    cols = cols[1:] + ['WILDCARD']
    df.columns = cols
    
    ## Preprocess data to handle it as text to facilitate joins
    for col in df:
        df[col] = df[col].str.strip()
        df[col] = df[col].str.upper()

    return df


def crear_conexion(db_file):
    """Crea una conexión a la base de datos SQLite especificada por db_file.
    :param db_file: Ruta del archivo de la base de datos.
    :return: Objeto de conexión o None.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(f"Error al conectar a la base de datos: {e}")
    return conn

# Crear la carpeta 'data/processed' si no existe
ruta_db = 'data/processed/reps_data.db'
os.makedirs(os.path.dirname(ruta_db), exist_ok=True)

### Preprocesamiento municipios

In [4]:
## Leer el dataframe
data_path = 'data/raw/Municipios.xlsx'
df_municipios = pd.read_excel(data_path)
df_municipios.head(2)

,DP,Departamento,MPIO,Municipio,Superficie,PopTot,Rural,Region
0,5,Ant%ioqUia,5001,Mede&l'lín,350.666623,2634570,1.6,Región Eje Cafetero
1,5,Ant%ioquia,5002,Abej!&orr*al,497.566212,21622,56.7,Región Eje Cafetero


In [15]:
## Preprocesar las columnas de departamento y municipio
for col in ['Departamento', 'Municipio']:
    df_municipios[col] = preprocess_text_column(df_municipios[col])

df_municipios.head(2)

,DP,Departamento,MPIO,Municipio,Superficie,PopTot,Rural,Region
0,05,Antioquia,05001,Medellín,350.666623,2634570,1.6,Región Eje Cafetero
1,05,Antioquia,05002,Abejorral,497.566212,21622,56.7,Región Eje Cafetero


In [16]:
## Fix MPIO codes
def fix_divipola_codes(series:pd.Series, fix_len:int=4) -> pd.Series:
    series = series.astype(str)
    series = series.apply(lambda t: f'0{t}' if len(t) == fix_len else t)
    return series

df_municipios['MPIO'] = fix_divipola_codes(df_municipios['MPIO'], fix_len=4)
df_municipios['DP'] = fix_divipola_codes(df_municipios['DP'], fix_len=1)

df_municipios.head(2)

,DP,Departamento,MPIO,Municipio,Superficie,PopTot,Rural,Region
0,05,Antioquia,05001,Medellín,350.666623,2634570,1.6,Región Eje Cafetero
1,05,Antioquia,05002,Abejorral,497.566212,21622,56.7,Región Eje Cafetero


In [17]:
## Enviar datos limpios a la db en data/processed

conn = crear_conexion(ruta_db)
df_municipios.to_sql(name='reps_municipios', con=conn, if_exists='replace', index=False)

1118

### Servicios preprocessing

In [7]:
## Leer la tabla de servicios
data_path = 'data/raw/Servicios.zip'
df_servicios = preprocess_read_reps_data(data_path)
df_servicios.head(2)

,DEPA_NOMBRE,MUNI_NOMBRE,HABI_CODIGO_HABILITACION,CODIGO_HABILITACION,NUMERO_SEDE,SEDE_NOMBRE,DIRECCION,TELEFONO,EMAIL,NITS_NIT,DV,CLASE_PERSONA,NAJU_CODIGO,NAJU_NOMBRE,CLPR_CODIGO,CLPR_NOMBRE,ESE,NIVEL,CARACTER,HABILITADO,GRSE_CODIGO,GRSE_NOMBRE,SERV_CODIGO,SERV_NOMBRE,AMBULATORIO,HOSPITALARIO,UNIDAD_MOVIL,DOMICILIARIO,OTRAS_EXTRAMURAL,CENTRO_REFERENCIA,INSTITUCION_REMISORA,COMPLEJIDAD_BAJA,COMPLEJIDAD_MEDIA,COMPLEJIDAD_ALTA,FECHA_APERTURA,FECHA_CIERRE,NUMERO_DISTINTIVO,NUMERO_SEDE_PRINCIPAL,OBSERVACIONES_SERV_RES3100_2019,FECHA_CORTE_REPS,NOMBRE,HORARIO_LUNES,HORARIO_MARTES,HORARIO_MIERCOLES,HORARIO_JUEVES,HORARIO_VIERNES,HORARIO_SABADO,HORARIO_DOMINGO,MODALIDAD_INTRAMURAL,MODALIDAD_EXTRAMURAL,MODALIDAD_UNIDAD_MOVIL,MODALIDAD_DOMICILIARIO,MODALIDAD_JORNADA_SALUD,MODALIDAD_TELEMEDICINA,MODALIDAD_PRESTADOR_REFERENCIA,MODALIDAD_PRESTADOR_REFERENCIA_TELEMEDICINA_INTERACTIVA,MODALIDAD_PRESTADOR_REFERENCIA_TELEMEDICINA_NO_INTERACTIVA,MODALIDAD_PRESTADOR_REFERENCIA_TELE_EXPERTICIA,MODALIDAD_PRESTADOR_REFERENCIA_TELE_MONITOREO,MODALIDAD_PRESTADOR_REMISOR,MODALIDAD_PRESTADOR_REMISOR_TELE_EXPERTICIA,MODALIDAD_PRESTADOR_REMISOR_TELE_MONITOREO,COMPLEJIDADES,ESPECIFICIDAD_ONCOLOGICO,ESPECIFICIDAD_TRASPLANTE_OSTEOMUSCULAR,ESPECIFICIDAD_TRASPLANTE_PIEL,ESPECIFICIDAD_TRASPLANTE_CARDIOVASCULAR,ESPECIFICIDAD_TRASPLANTE_TEJIDO_OCULAR,ESPECIFICIDAD_ATENCION_PACIENTE_QUEMADO,ESPECIFICIDAD_SALUD_MENTAL,ESPECIFICIDAD_SPA,ESPECIFICIDAD_OTRAS_PATOLOGIAS,ESPECIFICIDAD_TRASPLANTE_CELULAS_PROGENITORAS_HEMATOPOYETICAS,ESPECIFICIDAD_PROCEDIMIENTOS_QUIRURGICOS_AMBULATORIOS,ESPECIFICIDAD_ORGANO_RINON,ESPECIFICIDAD_ORGANO_HIGADO,ESPECIFICIDAD_ORGANO_PANCREAS,ESPECIFICIDAD_ORGANO_INTESTINO,ESPECIFICIDAD_ORGANO_MULTIVISCERAL,ESPECIFICIDAD_ORGANO_CORAZON,ESPECIFICIDAD_ORGANO_PULMON,ESPECIFICIDAD_SUSTANCIAS_PSICOACTIVAS,ESPECIFICIDAD_TRASPLANTE_RENAL,VERSION_NORMA,EMAIL_ADICIONAL,TELEFONO_ADICIONAL,GERENTE,MUNICIPIO PDET,MUNICIPIO ZOMAC,MUNICIPIO PNIS,MUNICIPIO PNSR ANTES 2023,MUNICIPIO PNSR 2023,MUNICIPIO PNSR 2024,WILDCARD
0,ANTIOQUIA,MEDELLÍN,0500100003,0500100003,01,BERNARDO AGUDELO JARAMILLO,CL 19A N° 44- 25 CONS 2301,3665349,CONSULTORIO2301@GMAIL.COM,71578430,0,NATURAL,1,PRIVADA,2,PROFESIONAL INDEPENDIENTE,,,NaN,SI,3,CONSULTA EXTERNA,320,GINECOBSTETRICIA,SI,NO,NO,NO,NO,NO,NO,NO,SI,NO,20110727,NaN,DHSS0107311,01,NaN,FECHA CORTE REPS: MAY 13 2025 12:58PM,BERNARDO AGUDELO JARAMILLO,08:00A17:00,NaN,NaN,NaN,08:00A17:00,NaN,NaN,SI,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,MEDIANA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,RESOLUCION_3100,NaN,NaN,NaN,NO,NO,NO,NO,NO,NO,NaN
1,ANTIOQUIA,MEDELLÍN,0500100008,0500100008,01,ANA CARMENZA POSADA ISAZA,CL 2 SUR # 46 - 55 CS 253 FASE II CLINICA LAS...,2687270,CORREOCLINICALASVEGAS@GMAIL.COM,32530725,0,NATURAL,1,PRIVADA,2,PROFESIONAL INDEPENDIENTE,,,NaN,SI,3,CONSULTA EXTERNA,328,MEDICINA GENERAL,SI,NO,NO,NO,NO,NO,NO,SI,NO,NO,20121121,NaN,DHSS0019899,01,NaN,FECHA CORTE REPS: MAY 13 2025 12:58PM,ANA CARMENZA POSADA ISAZA,08:00A12:00,08:00A18:00,08:00A12:00,08:00A18:00,08:00A12:00,NaN,NaN,SI,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,BAJA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,RESOLUCION_3100,NaN,NaN,NaN,NO,NO,NO,NO,NO,NO,NaN


In [8]:
## Enviar datos limpios a la db en data/processed
ruta_db = 'data/processed/reps_data.db'
conn = crear_conexion(ruta_db)

df_servicios.to_sql(name='reps_servicios', con=conn, if_exists='replace', index=False)

226641

### Sedes preprocessing (paso a paso)

In [9]:
## Read table as pandas dataframe
data_path = 'data/raw/Sedes.zip'
df_sedes = pd.read_csv(data_path, sep=';', encoding='ISO-8859-1', on_bad_lines='skip', dtype=str)
df_sedes.head(2)

,departamento,municipio,codigo_prestador,nombre_prestador,codigo_habilitacion,numero_sede,nombre,gerente,tipo_zona,direccion,barrio,cepo_codigo,centro_poblado,telefono,fax,email,fecha_apertura,fecha_cierre,nits_nit,dv,clase_persona,naju_codigo,naturaleza,clpr_codigo,clase_prestador,ese,nivel,caracter,sede_principal,habilitado,numero_sede_principal,horario_lunes,horario_martes,horario_miercoles,horario_jueves,horario_viernes,horario_sabado,horario_domingo,fecha_corte_REPS,telefono_adicional,email_adicional,Municipio PDET,Municipio ZOMAC,Municipio PNIS,Municipio PNSR antes 2023,Municipio PNSR 2023,Municipio PNSR 2024
Amazonas,EL ENCANTO,9100100019,E.S.E. HOSPITAL SAN RAFAEL DE LETICIA,9126300019,11,CENTRO DE SALUD SAN RAFAEL - E.S.E HOSPITAL SA...,DENNIS AMPARO VASQUEZ ARIAS,RURAL,CORREGIMIENTO DEL ENCANTO,NaN,91263000,EL ENCANTO,3203016139,NaN,sanrafael@esehospitalsanrafael-leticia-amazona...,20110331,NaN,838000096,7,JURIDICO,4,Pública,1,Instituciones Prestadoras de Servicios de Salu...,SI,2,DEPARTAMENTAL,NO,SI,01,07 a 18,07 a 18,07 a 18,07 a 18,07 a 18,00 a 24,00 a 24,Fecha corte REPS: May 13 2025 12:52PM,NaN,NaN,NO,NO,NO,SI,SI,SI,NaN
Amazonas,LA CHORRERA,9100100019,E.S.E. HOSPITAL SAN RAFAEL DE LETICIA,9140500019,14,CENTRO DE SALUD DE LA CHORRERA- E.S.E HOSPITAL...,DENNIS AMPARO VASQUEZ ARIAS,RURAL,CORREGIMIENTO DE LA CHORRERA,NaN,91405000,LA CHORRERA,3203016132,NaN,chorrera@esehospitalsanrafael-leticia-amazonas...,20110323,NaN,838000096,7,JURIDICO,4,Pública,1,Instituciones Prestadoras de Servicios de Salu...,SI,2,DEPARTAMENTAL,NO,SI,01,00 a 24,00 a 24,00 a 24,00 a 24,00 a 24,00 a 24,00 a 24,Fecha corte REPS: May 13 2025 12:52PM,NaN,NaN,NO,NO,NO,SI,SI,SI,NaN


In [10]:
## Alocate first colum from index
df_sedes = df_sedes.reset_index(names='wildcard')
df_sedes.head(2)

,wildcard,departamento,municipio,codigo_prestador,nombre_prestador,codigo_habilitacion,numero_sede,nombre,gerente,tipo_zona,direccion,barrio,cepo_codigo,centro_poblado,telefono,fax,email,fecha_apertura,fecha_cierre,nits_nit,dv,clase_persona,naju_codigo,naturaleza,clpr_codigo,clase_prestador,ese,nivel,caracter,sede_principal,habilitado,numero_sede_principal,horario_lunes,horario_martes,horario_miercoles,horario_jueves,horario_viernes,horario_sabado,horario_domingo,fecha_corte_REPS,telefono_adicional,email_adicional,Municipio PDET,Municipio ZOMAC,Municipio PNIS,Municipio PNSR antes 2023,Municipio PNSR 2023,Municipio PNSR 2024
0,Amazonas,EL ENCANTO,9100100019,E.S.E. HOSPITAL SAN RAFAEL DE LETICIA,9126300019,11,CENTRO DE SALUD SAN RAFAEL - E.S.E HOSPITAL SA...,DENNIS AMPARO VASQUEZ ARIAS,RURAL,CORREGIMIENTO DEL ENCANTO,NaN,91263000,EL ENCANTO,3203016139,NaN,sanrafael@esehospitalsanrafael-leticia-amazona...,20110331,NaN,838000096,7,JURIDICO,4,Pública,1,Instituciones Prestadoras de Servicios de Salu...,SI,2,DEPARTAMENTAL,NO,SI,01,07 a 18,07 a 18,07 a 18,07 a 18,07 a 18,00 a 24,00 a 24,Fecha corte REPS: May 13 2025 12:52PM,NaN,NaN,NO,NO,NO,SI,SI,SI,NaN
1,Amazonas,LA CHORRERA,9100100019,E.S.E. HOSPITAL SAN RAFAEL DE LETICIA,9140500019,14,CENTRO DE SALUD DE LA CHORRERA- E.S.E HOSPITAL...,DENNIS AMPARO VASQUEZ ARIAS,RURAL,CORREGIMIENTO DE LA CHORRERA,NaN,91405000,LA CHORRERA,3203016132,NaN,chorrera@esehospitalsanrafael-leticia-amazonas...,20110323,NaN,838000096,7,JURIDICO,4,Pública,1,Instituciones Prestadoras de Servicios de Salu...,SI,2,DEPARTAMENTAL,NO,SI,01,00 a 24,00 a 24,00 a 24,00 a 24,00 a 24,00 a 24,00 a 24,Fecha corte REPS: May 13 2025 12:52PM,NaN,NaN,NO,NO,NO,SI,SI,SI,NaN


In [11]:
## Clean columns names and assign correct order
cols = [c.upper().strip() for c in df_sedes.columns]

## Move WILCARD column to last position
cols = cols[1:] + ['WILDCARD']

df_sedes.columns = cols
df_sedes.head(2)

,DEPARTAMENTO,MUNICIPIO,CODIGO_PRESTADOR,NOMBRE_PRESTADOR,CODIGO_HABILITACION,NUMERO_SEDE,NOMBRE,GERENTE,TIPO_ZONA,DIRECCION,BARRIO,CEPO_CODIGO,CENTRO_POBLADO,TELEFONO,FAX,EMAIL,FECHA_APERTURA,FECHA_CIERRE,NITS_NIT,DV,CLASE_PERSONA,NAJU_CODIGO,NATURALEZA,CLPR_CODIGO,CLASE_PRESTADOR,ESE,NIVEL,CARACTER,SEDE_PRINCIPAL,HABILITADO,NUMERO_SEDE_PRINCIPAL,HORARIO_LUNES,HORARIO_MARTES,HORARIO_MIERCOLES,HORARIO_JUEVES,HORARIO_VIERNES,HORARIO_SABADO,HORARIO_DOMINGO,FECHA_CORTE_REPS,TELEFONO_ADICIONAL,EMAIL_ADICIONAL,MUNICIPIO PDET,MUNICIPIO ZOMAC,MUNICIPIO PNIS,MUNICIPIO PNSR ANTES 2023,MUNICIPIO PNSR 2023,MUNICIPIO PNSR 2024,WILDCARD
0,Amazonas,EL ENCANTO,9100100019,E.S.E. HOSPITAL SAN RAFAEL DE LETICIA,9126300019,11,CENTRO DE SALUD SAN RAFAEL - E.S.E HOSPITAL SA...,DENNIS AMPARO VASQUEZ ARIAS,RURAL,CORREGIMIENTO DEL ENCANTO,NaN,91263000,EL ENCANTO,3203016139,NaN,sanrafael@esehospitalsanrafael-leticia-amazona...,20110331,NaN,838000096,7,JURIDICO,4,Pública,1,Instituciones Prestadoras de Servicios de Salu...,SI,2,DEPARTAMENTAL,NO,SI,01,07 a 18,07 a 18,07 a 18,07 a 18,07 a 18,00 a 24,00 a 24,Fecha corte REPS: May 13 2025 12:52PM,NaN,NaN,NO,NO,NO,SI,SI,SI,NaN
1,Amazonas,LA CHORRERA,9100100019,E.S.E. HOSPITAL SAN RAFAEL DE LETICIA,9140500019,14,CENTRO DE SALUD DE LA CHORRERA- E.S.E HOSPITAL...,DENNIS AMPARO VASQUEZ ARIAS,RURAL,CORREGIMIENTO DE LA CHORRERA,NaN,91405000,LA CHORRERA,3203016132,NaN,chorrera@esehospitalsanrafael-leticia-amazonas...,20110323,NaN,838000096,7,JURIDICO,4,Pública,1,Instituciones Prestadoras de Servicios de Salu...,SI,2,DEPARTAMENTAL,NO,SI,01,00 a 24,00 a 24,00 a 24,00 a 24,00 a 24,00 a 24,00 a 24,Fecha corte REPS: May 13 2025 12:52PM,NaN,NaN,NO,NO,NO,SI,SI,SI,NaN


In [12]:
## Preprocess data to handle it as text to facilitate joins
for col in df_sedes:
    df_sedes[col] = df_sedes[col].str.strip()
    df_sedes[col] = df_sedes[col].str.upper()

df_sedes.tail(2)

,DEPARTAMENTO,MUNICIPIO,CODIGO_PRESTADOR,NOMBRE_PRESTADOR,CODIGO_HABILITACION,NUMERO_SEDE,NOMBRE,GERENTE,TIPO_ZONA,DIRECCION,BARRIO,CEPO_CODIGO,CENTRO_POBLADO,TELEFONO,FAX,EMAIL,FECHA_APERTURA,FECHA_CIERRE,NITS_NIT,DV,CLASE_PERSONA,NAJU_CODIGO,NATURALEZA,CLPR_CODIGO,CLASE_PRESTADOR,ESE,NIVEL,CARACTER,SEDE_PRINCIPAL,HABILITADO,NUMERO_SEDE_PRINCIPAL,HORARIO_LUNES,HORARIO_MARTES,HORARIO_MIERCOLES,HORARIO_JUEVES,HORARIO_VIERNES,HORARIO_SABADO,HORARIO_DOMINGO,FECHA_CORTE_REPS,TELEFONO_ADICIONAL,EMAIL_ADICIONAL,MUNICIPIO PDET,MUNICIPIO ZOMAC,MUNICIPIO PNIS,MUNICIPIO PNSR ANTES 2023,MUNICIPIO PNSR 2023,MUNICIPIO PNSR 2024,WILDCARD
74500,VICHADA,SANTA ROSALÍA,9900100006,E.S.E. HOSPITAL DEPARTAMENTAL SAN JUAN DE DIOS,9962400006,09,HOSPITAL SEDE SANTA ROSALIA,ALVARO ALBERTO CARDOSO CASTRO,URBANA,EL CENTRO,EL CENTRO,99624000,SANTA ROSALÍA,3102349354,NaN,CALIDAD@ESESANJUANDEDIOSPC.GOV.CO,20161205,NaN,842000004,4,JURIDICO,4,PÚBLICA,1,INSTITUCIONES PRESTADORAS DE SERVICIOS DE SALU...,SI,2,DEPARTAMENTAL,NO,SI,01,00 A 24,00 A 24,00 A 24,00 A 24,00 A 24,00 A 24,00 A 24,FECHA CORTE REPS: MAY 13 2025 12:52PM,NaN,NaN,NO,NO,NO,SI,SI,SI,NaN
74501,VICHADA,SANTA ROSALÍA,9962400150,JOSE OMAR OSPINA JAIMES,9962400150,01,JOSE OMAR OSPINA JAIMES,NaN,URBANA,CRA 8 # 5 - 75,CENTRO,,NaN,3214665867,NaN,OSPINAJAIMESJOSEOMAR@GMAIL.COM,20211011,NaN,1121844096,NaN,NATURAL,1,PRIVADA,2,PROFESIONAL INDEPENDIENTE,,,NaN,SI,SI,01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FECHA CORTE REPS: MAY 13 2025 12:52PM,3202901466,JOSEOSPINA21@HOTMAIL.COM,NO,NO,NO,SI,SI,SI,NaN


In [13]:
## Load table to database
ruta_db = 'data/processed/reps_data.db'
conn = crear_conexion(ruta_db)

df_sedes.to_sql(name='reps_sedes', con=conn, if_exists='replace', index=False)

74502

### Generación de query para extraer información de la base de datos

Para este ejercicio específico, usando las columnas 'ESPECIFICIDAD_ONCOLOGICO' y 'ESPECIFICIDAD_TRASPLANTE_RENAL' se quiere generar un dato de la cantidad de sedes que tienen la marcación 'SI' en estas columnas y generar una cantidad por cada 100.000 habitantes para regiones y una por cada 10.000 habitantes para municipios. 

In [64]:
# Generate the sql query
query = (
    f"""
    -- Crear cte para procesar tablas y facilitar la lectura del código
    
    -- traer la tabla de servicios oncologicos y la cantidad de sedes por 
    -- municipios.
    WITH servicios AS (
        SELECT CODIGO_HABILITACION
            , COMPLEJIDADES
            , SERV_NOMBRE
            , ESPECIFICIDAD_ONCOLOGICO
            , ESPECIFICIDAD_TRASPLANTE_RENAL 
            , SUBSTRING(CODIGO_HABILITACION, 1, 5) AS COD_MUNICIPIO
            , CODIGO_HABILITACION || NUMERO_SEDE AS COD_SEDE

        FROM reps_servicios
        WHERE  1 = 1
            AND HABILITADO = 'SI'
    )
    
    -- Traer la tabla de municipios
    , municipios AS (
        SELECT MPIO AS COD_MUNICIPIO
            , Municipio
            , Departamento
            , Region
            , PopTot
        FROM reps_municipios
    )
    
    -- Agrupar la tabla por servicios para tener conteos por tipo de servicio
    , servicios_oncologico AS (
        SELECT COD_MUNICIPIO, COUNT(DiSTINCT COD_SEDE) AS SEDES_ONCOLOGICO
            , 'ONCOLOGICO' AS tipo_servicio
        FROM servicios
        WHERE 1 = 1
            AND ESPECIFICIDAD_ONCOLOGICO = 'SI'
        
        GROUP BY COD_MUNICIPIO
    )
    , servicios_renal AS (
        SELECT COD_MUNICIPIO, COUNT(DiSTINCT COD_SEDE) AS SEDES_RENAL
            , 'ONCOLOGICO' AS tipo_servicio
        FROM servicios
        WHERE 1 = 1
            AND ESPECIFICIDAD_TRASPLANTE_RENAL = 'SI'
        
        GROUP BY COD_MUNICIPIO
    )

    -- Hacer la unión con la tabla de municipios para hallar la cantidad de sedes
    -- para las dos especialidades específicas por municipio
    SELECT municipios.COD_MUNICIPIO
        , municipios.Municipio
        , municipios.Departamento
        , municipios.Region
        , municipios.PopTot
        , COALESCE(SEDES_ONCOLOGICO, 0) AS SEDES_ONCOLOGICO
        , COALESCE(SEDES_RENAL, 0) AS SEDES_RENAL
    FROM municipios
    LEFT JOIN servicios_oncologico 
    ON servicios_oncologico.COD_MUNICIPIO = municipios.COD_MUNICIPIO
    
    LEFT JOIN servicios_renal 
    ON servicios_renal.COD_MUNICIPIO = municipios.COD_MUNICIPIO
    
    """
)

## Use pandas to query de reps_sedes database
df = pd.read_sql_query(query, conn)
df.to_csv('data/processed/conteos_sedes.csv', sep='|', index=False)

,COD_MUNICIPIO,Municipio,Departamento,Region,PopTot,SEDES_ONCOLOGICO,SEDES_RENAL
0,05001,Medellín,Antioquia,Región Eje Cafetero,2634570,38,3
1,05002,Abejorral,Antioquia,Región Eje Cafetero,21622,0,0
2,05004,Abriaquí,Antioquia,Región Eje Cafetero,2872,0,0
3,05021,Alejandría,Antioquia,Región Eje Cafetero,4989,0,0
4,05030,Amagá,Antioquia,Región Eje Cafetero,32628,0,0
...,...,...,...,...,...,...,...
1113,97889,Yavaraté,Vaupés,Región Llano,1265,0,0
1114,99001,Puerto Carreño,Vichada,Región Llano,22963,0,0
1115,99524,La Primavera,Vichada,Región Llano,11380,0,0
1116,99624,Santa Rosalía,Vichada,Región Llano,4732,0,0
